In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
articles_df = pd.read_csv("articles_subset.csv")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
articles_df.isna().count()

article_id                      99773
product_code                    99773
prod_name                       99773
product_type_no                 99773
product_type_name               99773
product_group_name              99773
graphical_appearance_no         99773
graphical_appearance_name       99773
colour_group_code               99773
colour_group_name               99773
perceived_colour_value_id       99773
perceived_colour_value_name     99773
perceived_colour_master_id      99773
perceived_colour_master_name    99773
department_no                   99773
department_name                 99773
index_code                      99773
index_name                      99773
index_group_no                  99773
index_group_name                99773
section_no                      99773
section_name                    99773
garment_group_no                99773
garment_group_name              99773
detail_desc                     99773
dtype: int64

In [ ]:
new_articles = articles_df.iloc[:,[0,2,4,5,7,9,11,13,15,17,19,21,23,24]]

In [ ]:
new_articles.shape

(99773, 14)

## Creating Features for Item Profiles

In [ ]:

new_articles["detail_desc"] = new_articles["detail_desc"].fillna('')

In [ ]:
def combine_features(row):
    try:
        #return '_'.join(row['prod_name'].split(" ")) +" "+'_'.join(row['product_type_name'].split(" "))+" "+'_'.join(row["product_group_name"].split(" "))+" "+'_'.join(row["graphical_appearance_name"].split(" "))+" "+'_'.join(row["colour_group_name"].split(" ")) + " "+ '_'.join(row['perceived_colour_value_name'].split(" ")) +" "+'_'.join(row['perceived_colour_master_name'].split(" "))+" "+'_'.join(row["department_name"].split(" "))+" "+'_'.join(row["index_name"].split(" "))+" "+'_'.join(row["index_group_name"].split(" ")) +" "+ '_'.join(row["section_name"].split(" "))+" "+'_'.join(row["garment_group_name"].split(" "))  
        return row['prod_name'] +" "+row['product_type_name']+" "+row["product_group_name"]+" "+row["graphical_appearance_name"]+" "+row["colour_group_name"] + " "+ row['perceived_colour_value_name'] +" "+row['perceived_colour_master_name']+" "+row["department_name"]+" "+row["index_name"]+" "+row["index_group_name"] +" "+ row["section_name"]+" "+row["garment_group_name"]+" "+row["detail_desc"]
    except Exception as e:
        print("Error:", e,row)
    
new_articles["combined_features"] = new_articles.apply(combine_features,axis=1)

In [ ]:
df_new = new_articles
df_new['index'] = df_new.index

In [ ]:
stop_words = stopwords.words('english')
df_new['combined_features'] = df_new['combined_features'].str.lower().str.split()
df_new["features"]=df_new["combined_features"].apply(lambda x: [word for word in x if word not in stop_words])
df_new["features"]=df_new["features"].apply(lambda x: " ".join(x))

In [ ]:
df_new.head()

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc,combined_features,index,features
0,108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,"[strap, top, vest, top, garment, upper, body, ...",0,strap top vest top garment upper body solid bl...
1,108775044,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,"[strap, top, vest, top, garment, upper, body, ...",1,strap top vest top garment upper body solid wh...
2,108775051,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,"[strap, top, (1), vest, top, garment, upper, b...",2,strap top (1) vest top garment upper body stri...
3,110065001,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...","[op, t-shirt, (idro), bra, underwear, solid, b...",3,op t-shirt (idro) bra underwear solid black da...
4,110065002,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...","[op, t-shirt, (idro), bra, underwear, solid, w...",4,op t-shirt (idro) bra underwear solid white li...


In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df_new["features"])

In [ ]:
item_features_df = pd.DataFrame(count_matrix.toarray(), columns = cv.get_feature_names_out(),index = df_new['article_id'])

In [ ]:
item_features = count_matrix.toarray()

In [ ]:
#cosine_sim = cosine_similarity(count_matrix)

In [ ]:
def get_title_from_index(index):
    return df_new[df_new.index == index]["article_id"].values[0]

def get_home(index):
    return df_new[df_new.index == index]["prod_name"].values[0],df_new[df_new.index == index]["colour_group_name"].values[0]

def get_index_from_title(title):
    return df_new[df_new.article_id == title]["index"].values[0]

## Creating User Profiles  From Item Profiles

In [ ]:
customers_df = pd.read_csv("customers_subset.csv")

In [ ]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,1.0,1.0,ACTIVE,Regularly,56.0,b31984b20a8c478de38eaf113c581ff64e63c4242e607b...
3,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,1.0,1.0,ACTIVE,Regularly,30.0,7c902dca60ee0bd0f9030eefd445d11146e8d24835738a...
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,1.0,1.0,ACTIVE,Regularly,44.0,930b19ae7db8abb5a27f4da10217755a7305b4c452f5e0...


In [ ]:
transactions_df = pd.read_csv("transactions_train_subset.csv").drop(['t_dat','price','sales_channel_id'],axis=1)

In [ ]:
transactions_df

,customer_id,article_id
0,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001
...,...,...
13310255,ffe1791c9c6e3df9aafeebc77cf2cf03dd0123ac76ef90...,750422018
13310256,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,853316001
13310257,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,296366006
13310258,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,789769001


In [ ]:
transactions_df = transactions_df[transactions_df.customer_id.isin(pd.read_csv("transactions_test_subset.csv").customer_id)]

In [ ]:
transactions_df

,customer_id,article_id
45,00d781e94d9a533dceb85a8d50a898d5987ad9921ea8c6...,685687004
46,00d781e94d9a533dceb85a8d50a898d5987ad9921ea8c6...,678983004
47,00d781e94d9a533dceb85a8d50a898d5987ad9921ea8c6...,616708004
48,00d781e94d9a533dceb85a8d50a898d5987ad9921ea8c6...,568858001
49,00d781e94d9a533dceb85a8d50a898d5987ad9921ea8c6...,640462002
...,...,...
13310254,ffe1791c9c6e3df9aafeebc77cf2cf03dd0123ac76ef90...,750422018
13310255,ffe1791c9c6e3df9aafeebc77cf2cf03dd0123ac76ef90...,750422018
13310256,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,853316001
13310257,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,296366006


In [ ]:
l = []

for i in range(5):
  l.append(transactions_df.iloc[i*582227:(i+1)*582227].merge(item_features_df,on= "article_id",how="left"))


In [ ]:
merged_trans = pd.concat(l)

In [ ]:
user_feature_df = merged_trans.drop(['article_id'],axis=1).groupby('customer_id').mean()

In [ ]:
user_feature_df.shape

In [ ]:
item_features_df.shape

In [ ]:
item_user_features = pd.concat([item_features_df,user_feature_df])

## Generating Recommendations 

In [ ]:
item_user_sim = cosine_similarity(item_user_features.to_numpy())

In [ ]:
all_recommendations = []

for idx,customer_id_iter in enumerate(user_feature_df.index):
  customer_idx = 99773 + idx
  similar_products =  list(enumerate(item_user_sim[customer_idx][:99773]))

  sorted_similar_products = sorted(similar_products,key=lambda x:x[1],reverse=True)

  temp = []
  i=0
  for element in sorted_similar_products:
    #print(element,i)
    try:
     # print (get_title_from_index(element[0]),get_home(element[0]))
      if int(element[0]) < 99773 :
        temp.append(get_title_from_index(element[0]))
        i=i+1
    except Exception:
      continue
    if i>12:
      break
  all_recommendations.append(temp)


In [ ]:
len(all_recommendations)

In [ ]:
user_feature_df["article_id"] = all_recommendations

In [ ]:
user_feature_df[['customer_id','article_id']].to_csv("content_based_recommender.csv")